In [3]:
%pip install pandas
%pip install scikit-learn
%pip install plotly
%pip install numpy
%pip install matplotlib
%pip install nbformat
%pip install opencv-python
%pip install tensorflow
%pip install wandb
%pip install imblearn

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import plotly.express as px
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import F1Score, CategoricalAccuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from imblearn.over_sampling import SMOTE

import wandb
%env "WANDB_NOTEBOOK_NAME" "model_balanced.ipynb"

wandb.login()

from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

env: "WANDB_NOTEBOOK_NAME"="model_balanced.ipynb"


In [6]:
label_df = pd.read_csv(
    "../../../data/processed/combined/combined_label.csv", index_col=False
)
data_dir = "../../../data/processed/combined/img/"

label_df.head()

,image_name,expression_label
0,angry_0.jpg,angry
1,angry_1.jpg,angry
2,fear_2.jpg,fear
3,angry_4.jpg,angry
4,angry_5.jpg,angry


In [7]:
def load_data(dir_path, img_size):
    X = []
    y = []

    for index, row in label_df.iterrows():
        image_path = os.path.join(dir_path, row["image_name"])
        img = cv2.imread(image_path)
        img = (
            cv2.resize(img, img_size, interpolation=cv2.INTER_AREA).astype("float32")
            / 255
        )
        X.append(img)
        y.append(row["expression_label"])

    X = np.array(X)
    y = np.array(y)
    return X, y

In [8]:
IMG_SIZE = (96, 96)

X, y = load_data(data_dir, IMG_SIZE)

In [9]:
dummies = pd.get_dummies(label_df["expression_label"])
dummies.head()

,angry,disgust,fear,happy,neutral,sad,surprise
0,True,False,False,False,False,False,False
1,True,False,False,False,False,False,False
2,False,False,True,False,False,False,False
3,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False


In [10]:
y = dummies.values
y

array([[ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False]])

In [11]:
train_ratio = 0.80
test_ratio = 0.10
validation_ratio = 0.10


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=validation_ratio / (train_ratio + test_ratio)
)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(63451, 96, 96, 3)
(7932, 96, 96, 3)
(7932, 96, 96, 3)
(63451, 7)
(7932, 7)
(7932, 7)


# Distribution of classes in each set


In [12]:
class_count_in_train_set = np.sum(y_train, axis=0)

print("Class distribution:")
print(class_count_in_train_set)

Class distribution:
[ 6888  3634  4973 12632 13202 13326  8796]


In [13]:
labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
fig = px.bar(x=labels, y=class_count_in_train_set, color=labels)

fig.update_layout(
    title="Distribution of each label in train set",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [17]:
X_train.shape

(63451, 96, 96, 3)

In [18]:
# get all dimensions of the resulting X
n_samples, height, width, n_channels = [X_train.shape[index] for index in range(4)]

# reshape X because SMOTE accepts only (n_samples, n_channels*height*weight)-type data
X_train_reshaped = X_train.reshape(n_samples, n_channels*height*width)

In [19]:
# initialize the SMOTE model
smote = SMOTE(random_state=62)

# perform re-sampling on modified X given y
X_train_smote, y_train_smote = smote.fit_resample(X_train_reshaped, y_train)

Exception in thread Thread-28 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\Jonas\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\Emotion_Recognition\.venv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\Jonas\AppData\Local\Programs\Python\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Jonas\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 1494, in _readerthread
    buffer.append(fh.read())
  File "C:\Users\Jonas\AppData\Local\Programs\Python\Python310\lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 3: character maps to <undefined>
c:\Emotion_Recognition\.venv\lib\site-packages\joblib\externals\loky\backend\context.py

In [20]:
class_count_in_train_smote_set = np.sum(y_train_smote, axis=0)

print("Class distribution:")
print(class_count_in_train_set)

Class distribution:
[ 6888  3634  4973 12632 13202 13326  8796]


In [21]:
labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
fig = px.bar(x=labels, y=class_count_in_train_smote_set, color=labels)

fig.update_layout(
    title="Distribution of each label in train set after SMOTE",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [22]:
class_count_in_validation_set = np.sum(y_valid, axis=0)

print("Class distribution:")
print(class_count_in_validation_set)

Class distribution:
[ 889  442  616 1584 1634 1632 1135]


In [23]:
labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
fig = px.bar(x=labels, y=class_count_in_validation_set, color=labels)

fig.update_layout(
    title="Distribution of each label in validation set",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [24]:
class_count_in_test_set = np.sum(y_test, axis=0)

print("Class distribution:")
print(class_count_in_test_set)

Class distribution:
[ 847  466  620 1594 1596 1678 1131]


In [25]:
labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
fig = px.bar(x=labels, y=class_count_in_test_set, color=labels)

fig.update_layout(
    title="Distribution of each label in test set",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [32]:
X_train_smote = X_train_smote.reshape(len(X_train_smote), 96, 96, 3)

In [43]:
X_train_smote.shape

(93282, 96, 96, 3)

In [41]:
y_train_smote.shape

(93282, 7)

# Data augmentation to balance the dataset


In [31]:
# # Assume x_train and y_train are your original data
# # Create an ImageDataGenerator with desired augmentations
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True,
#     brightness_range=(0.8, 1.2),
#     zoom_range=0.1,
# )

# # Generate augmented images for the minority class
# angry_indices = np.where(y_train_smote[:, 0] == 1)[0]
# disgust_indices = np.where(y_train_smote[:, 1] == 1)[0]
# fear_indices = np.where(y_train_smote[:, 2] == 1)[0]
# augmented_angry_images = datagen.flow(
#     X_train_smote[angry_indices], y_train_smote[angry_indices], batch_size=len(angry_indices)
# )
# augmented_disgust_images = datagen.flow(
#     X_train_smote[disgust_indices], y_train_smote[disgust_indices], batch_size=len(disgust_indices)
# )
# augmented_fear_images = datagen.flow(
#     X_train_smote[fear_indices], y_train_smote[fear_indices], batch_size=len(fear_indices)
# )

# # Combine augmented minority class images with original data
# x_train_balanced = np.concatenate(
#     [
#         X_train_smote,
#         augmented_angry_images[0][0],
#         augmented_disgust_images[0][0],
#         augmented_fear_images[0][0],
#     ]
# )
# y_train_balanced = np.concatenate(
#     [
#         y_train_smote,
#         augmented_angry_images[0][1],
#         augmented_disgust_images[0][1],
#         augmented_fear_images[0][1],
#     ]
# )

In [27]:
class_count_in_balanced_train_set = np.sum(y_train_balanced, axis=0)

print("Class distribution:")
print(class_count_in_balanced_train_set)

Class distribution:
[13776  7268  9946 12632 13202 13326  8796]


In [19]:
labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
fig = px.bar(x=labels, y=class_count_in_balanced_train_set, color=labels)

fig.update_layout(
    title="Distribution of each label in balanced train set",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

# Model training


In [35]:
IMG_SIZE = (96, 96)

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=IMG_SIZE + (3,))
# for layer in base_model.layers[:-1]:
#     layer.trainable = False


base_model.trainable = False

In [36]:
NUM_CLASSES = 7

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(NUM_CLASSES, activation="softmax"))

In [46]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Emotion Recognition",
    # track hyperparameters and run metadata with wandb.config
    config={
        "optimizer": "adam",
        "loss": "categorical_crossentropy",
        "metric": ["categorical_accuracy"],
        "epoch": 5,
        "batch_size": 128,
        "architecture": "MobileNetV2",
        "dropout": 0.4,
        "learning_rate": 0.0003,
        "Dense": 512,
        "IMG_SIZE": "96, 96",
        #"initial_training": "5 Epochs",
        #"retraining": "20 epochs"
    },
)

In [47]:
config = wandb.config


model.compile(
    loss=config["loss"],
    optimizer=Adam(learning_rate = 0.0003),
    metrics=config["metric"],

)

history = model.fit(
    X_train_smote,
    y_train_smote,
    validation_data=(X_valid, y_valid),
    epochs=config["epoch"],
    batch_size=config["batch_size"],
    callbacks=[WandbMetricsLogger(log_freq=5)],
)


wandb.finish()

MemoryError: Unable to allocate 9.61 GiB for an array with shape (93282, 96, 96, 3) and data type float32

In [21]:
base_model.trainable = True

In [22]:
model.compile(
    loss=config["loss"],
    optimizer=Adam(learning_rate = 0.0003),
    metrics=config["metric"],

)

history = model.fit(
    x_train_balanced,
    y_train_balanced,
    validation_data=(X_valid, y_valid),
    epochs= 20,
    batch_size=config["batch_size"],
    callbacks=[WandbMetricsLogger(log_freq=5)],
)

wandb.finish()

Epoch 1/20
617/617 ━━━━━━━━━━━━━━━━━━━━ 283s 453ms/step - categorical_accuracy: 0.4985 - loss: 1.2395 - val_categorical_accuracy: 0.4570 - val_loss: 1.4566
Epoch 2/20
617/617 ━━━━━━━━━━━━━━━━━━━━ 257s 417ms/step - categorical_accuracy: 0.5187 - loss: 1.1926 - val_categorical_accuracy: 0.4603 - val_loss: 1.4583
Epoch 3/20
617/617 ━━━━━━━━━━━━━━━━━━━━ 249s 404ms/step - categorical_accuracy: 0.5324 - loss: 1.1496 - val_categorical_accuracy: 0.4532 - val_loss: 1.4630
Epoch 4/20
617/617 ━━━━━━━━━━━━━━━━━━━━ 248s 401ms/step - categorical_accuracy: 0.5473 - loss: 1.1144 - val_categorical_accuracy: 0.4574 - val_loss: 1.4946
Epoch 5/20
617/617 ━━━━━━━━━━━━━━━━━━━━ 245s 397ms/step - categorical_accuracy: 0.5665 - loss: 1.0647 - val_categorical_accuracy: 0.4578 - val_loss: 1.5183
Epoch 6/20
617/617 ━━━━━━━━━━━━━━━━━━━━ 242s 392ms/step - categorical_accuracy: 0.5850 - loss: 1.0243 - val_categorical_accuracy: 0.4555 - val_loss: 1.5591
Epoch 7/20
617/617 ━━━━━━━━━━━━━━━━━━━━ 252s 409ms/step - catego

batch/batch_step,▁▁▁▂▂▂▂▃▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
batch/categorical_accuracy,▁▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████
batch/loss,█▇▆▆▆▅▅▅▅▅▄▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁
epoch/categorical_accuracy,▁▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇▇▇████
epoch/epoch,▁▁▂▂▂▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/loss,█▇▇▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁
epoch/val_categorical_accuracy,▁▂▅▅▅▇▇▆▇▇▇█▇▇▇▇▆▆▆▆▆▆▅▆▅
epoch/val_loss,▂▂▁▁▁▁▁▁▁▂▂▃▂▃▄▄▄▄▆▆▇▇▇██
batch/batch_step,12395
batch/categorical_accuracy,0.71781
batch/loss,0.66214
